In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.metrics.pairwise import cosine_similarity
import os

In [4]:
import pickle
features_file = 'features.pkl'

def save_features(features, filename):
    with open(filename, 'wb') as f:
        pickle.dump(features, f)
def load_features(filename):
    with open(filename, 'rb') as f:
        features = pickle.load(f)
    return features


In [5]:
base_model = ResNet50(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

In [6]:
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

In [7]:
def extract_features(img_path):
    img_array = preprocess_image(img_path)
    features = model.predict(img_array)
    return features


In [8]:
def compute_similarity(feature1, feature2):
    similarity = cosine_similarity(feature1.reshape(1, -1), feature2.reshape(1, -1))
    return similarity[0][0]


In [9]:
def get_images_features():
    if os.path.exists(features_file) :
        return load_features(features_file)    
    else:
        dataset_path = os.path.join(os.path.dirname(os.getcwd()),'server','data','images')

        image_features = {}
        for img_file in os.listdir(dataset_path):
           for img_name in os.listdir(os.path.join(dataset_path,img_file)): 
            img_path = os.path.join(dataset_path,img_file, img_name)
            features = extract_features(img_path)
            image_features[img_name] = features
            save_features(image_features,features_file)
        return image_features


In [10]:
input_image_path =''

input_image_features = extract_features(input_image_path)

1/1 [==============================] - 3s 3s/step


In [11]:

def get_similarities(input_image_features):
    similarities = {}
    for img_name, features in get_images_features().items():
        similarity = compute_similarity(input_image_features, features)

        if(similarity>0.60):
           similarities[img_name] = similarity
        if(len(similarities)>=20):
               break
    sorted_images = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    return sorted_images    

In [20]:
def go(image_url):
    input_image_features=extract_features(image_url)
    res=get_similarities(input_image_features)
    similarities=[[img[0],img[1]] for img in res]
    print(similarities)
    return similarities

In [21]:
go('C:/Users/sami/Desktop/projects/2syria/server/data/images/ChIJba2CbpuuJhUR5Td9vMgQrS4/AWU5eFhMp2_qiQ0vBa5WSfSVN-q6zkqzUJqUKOsuf0sVXcchLS_LUbd1bXgyuqkbUoA03QC5lO_vRsGSIcPvYs-n9VquHYvPO2RZX8qq8RoelblRD2tqgyv-E7K6VPMrsY7p8WluI7a6RmRWtf8BCoPW2zm19P6zkUkOtBJFV5YBsXxwqaSF.jpg')


1/1 [==============================] - 0s 262ms/step
[['AWU5eFhMp2_qiQ0vBa5WSfSVN-q6zkqzUJqUKOsuf0sVXcchLS_LUbd1bXgyuqkbUoA03QC5lO_vRsGSIcPvYs-n9VquHYvPO2RZX8qq8RoelblRD2tqgyv-E7K6VPMrsY7p8WluI7a6RmRWtf8BCoPW2zm19P6zkUkOtBJFV5YBsXxwqaSF.jpg', 0.99999994], ['AWU5eFgruMSYk48A1bAi6QqA-SRivsIgvKjfJPrFO42hv16xmCWAOLPyugtEY2prqecNmoOgT0c8buz1YupZoqfPWWsILf8RpaRoQQTOwcVPIBAiCBWSjnkGrEUV5UgKRBMFy0LkD1dWXAolWZojtKhxHBFyyi999Dqg1lYmPZqMgoiXMiwR.jpg', 0.6988811], ['AWU5eFgIfwYHEOYpKU05HQQ5w0_NsvWXu5vxmgPR6RdZEeb_BcC-sRtcoG9XeZ5bKEYBnXVdfShla7YSE1iZV-S2sx-zARVm0w5NuoLZ12brbAeA3JnnTZ7y4Ke7xQiqrviOMGp5cvHx8nNSkWDVLnedJQttTAGYvmNnZ809RWVdSTJyuqkl.jpg', 0.68572575], ['AWU5eFgaxkEttoX7re7ubyA8KAQyJLutS0ZRcZ9TqT3CVORWNh1ey7P91zB49d0AKUqaOz2p_1ti-VwV4ZBK5_3Ik11JAxTG1aGk1wJqdNPOtjdXWLc-Y4p_075U1hNBOBl7iQZ8goMh504eNlojkH1gSBSqomVPJPMewH1hHnjewDsdlmU-.jpg', 0.68496263], ['AWU5eFg-k5HRiOim9Awyv67NUjEB5FzliaoifnaRgxTsxBDjwRcILSB-QmQ0WSqt5vuJeMRqcf-jZKGwY2CEdA4O-P9O2Vv1B-OZhJ0SiiLyeiKu-AWD_-dgXXqZq_fJ6fyr7beQy

[['AWU5eFhMp2_qiQ0vBa5WSfSVN-q6zkqzUJqUKOsuf0sVXcchLS_LUbd1bXgyuqkbUoA03QC5lO_vRsGSIcPvYs-n9VquHYvPO2RZX8qq8RoelblRD2tqgyv-E7K6VPMrsY7p8WluI7a6RmRWtf8BCoPW2zm19P6zkUkOtBJFV5YBsXxwqaSF.jpg',
  0.99999994],
 ['AWU5eFgruMSYk48A1bAi6QqA-SRivsIgvKjfJPrFO42hv16xmCWAOLPyugtEY2prqecNmoOgT0c8buz1YupZoqfPWWsILf8RpaRoQQTOwcVPIBAiCBWSjnkGrEUV5UgKRBMFy0LkD1dWXAolWZojtKhxHBFyyi999Dqg1lYmPZqMgoiXMiwR.jpg',
  0.6988811],
 ['AWU5eFgIfwYHEOYpKU05HQQ5w0_NsvWXu5vxmgPR6RdZEeb_BcC-sRtcoG9XeZ5bKEYBnXVdfShla7YSE1iZV-S2sx-zARVm0w5NuoLZ12brbAeA3JnnTZ7y4Ke7xQiqrviOMGp5cvHx8nNSkWDVLnedJQttTAGYvmNnZ809RWVdSTJyuqkl.jpg',
  0.68572575],
 ['AWU5eFgaxkEttoX7re7ubyA8KAQyJLutS0ZRcZ9TqT3CVORWNh1ey7P91zB49d0AKUqaOz2p_1ti-VwV4ZBK5_3Ik11JAxTG1aGk1wJqdNPOtjdXWLc-Y4p_075U1hNBOBl7iQZ8goMh504eNlojkH1gSBSqomVPJPMewH1hHnjewDsdlmU-.jpg',
  0.68496263],
 ['AWU5eFg-k5HRiOim9Awyv67NUjEB5FzliaoifnaRgxTsxBDjwRcILSB-QmQ0WSqt5vuJeMRqcf-jZKGwY2CEdA4O-P9O2Vv1B-OZhJ0SiiLyeiKu-AWD_-dgXXqZq_fJ6fyr7beQy7k3tdcvWIvm59tdRCEYK7EXcjoZCMPlqwUgEMFCSc